# Advent of Code 2024

## Day 0: Imports and Utility Functions

In [1]:
from collections import Counter
from typing import List

In [2]:
def file_to_list(filename, sep="\n", maxsplit=-1) -> List[str]:
    """
    Read an input file and split it using sep as the delimiter.
    """
    with open(filename) as f:
        return f.read().rstrip().split(sep, maxsplit=maxsplit)

## Day 1: Historian Hysteria

### Part 1

In [3]:
def day1part1():
    input_lines = file_to_list("./inputs/input1.txt")
    list_1 = []
    list_2 = []
    for line in input_lines:
        val1, val2 = line.split("   ")
        list_1.append(int(val1))
        list_2.append(int(val2))
    list_1.sort()
    list_2.sort()
    distance = 0
    for x, y in zip(list_1, list_2):
        distance += abs(x - y)
    return(distance)

day1part1()

2904518

### Part 2

In [4]:
def day1part2():
    input_lines = file_to_list("./inputs/input1.txt")
    list_1 = []
    list_2 = []
    for line in input_lines:
        val1, val2 = line.split("   ")
        list_1.append(int(val1))
        list_2.append(int(val2))
    counts = Counter(list_2)
    sim_score = sum([(val * counts.get(val, 0)) for val in list_1])
    return(sim_score)

day1part2()

18650129